# IG to Test Kit FULL pipeline

## Setup

### Importing Notebooks as Modules (from the [Jupyter Notebook Documentation](https://jupyter-notebook.readthedocs.io/en/4.x/examples/Notebook/rstversions/Importing%20Notebooks.html))

In [44]:
import inspect
import json
import llm_utils
import importlib
import tiktoken

## Initializing LLM Clients

In [45]:
importlib.reload(llm_utils)
llm_clients = llm_utils.LLMApiClient()

In [46]:
llm_clients.clients

{'claude': <anthropic.Anthropic at 0x32b1104d0>,
 'gemini': genai.GenerativeModel(
     model_name='models/gemini-2.5-flash',
     generation_config={'max_output_tokens': 8192, 'temperature': 0.3},
     safety_settings={<HarmCategory.HARM_CATEGORY_HARASSMENT: 7>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_HATE_SPEECH: 8>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: 9>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 10>: <HarmBlockThreshold.BLOCK_NONE: 4>},
     tools=None,
     system_instruction=None,
     cached_content=None
 ),
 'gpt': <openai.OpenAI at 0x33339bf80>}

## Text Extraction

### HTML to Markdown Conversion Using Markdownify (Langchain Tool)

In [4]:
import HTML_extractor

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
urls = [
    "https://hl7.org/fhir/us/davinci-pdex-plan-net/index.html",
    "https://hl7.org/fhir/us/davinci-pdex-plan-net/ChangeHistory.html",
    "https://hl7.org/fhir/us/davinci-pdex-plan-net/examples.html",
    "https://hl7.org/fhir/us/davinci-pdex-plan-net/implementation.html",
    "https://hl7.org/fhir/us/davinci-pdex-plan-net/profiles.html",
    "https://hl7.org/fhir/us/davinci-pdex-plan-net/artifacts.html",
    "https://hl7.org/fhir/us/davinci-pdex-plan-net/CapabilityStatement-plan-net.html"
]

HTML_extractor.convert_urls_to_markdown(urls, output_dir="text_extraction/PlanNet/site/markdown")

Fetching pages: 100%|##########| 7/7 [00:00<00:00,  7.93it/s]


Created: index.md
Created: ChangeHistory.md
Created: examples.md
Created: implementation.md
Created: profiles.md
Created: artifacts.md
Created: CapabilityStatement_plan_net.md


### Markdown Post-processing

In [7]:
import markdown_cleaner
markdown_cleaner.process_directory("checkpoints/text_extraction/PlanNet/site/markdown", "checkpoints/post_processing/")

Found 7 markdown files in checkpoints/text_extraction/PlanNet/site/markdown
Cleaned and saved: checkpoints/post_processing/implementation.md
Cleaned and saved: checkpoints/post_processing/examples.md
Cleaned and saved: checkpoints/post_processing/profiles.md
Cleaned and saved: checkpoints/post_processing/ChangeHistory.md
Cleaned and saved: checkpoints/post_processing/artifacts.md
Cleaned and saved: checkpoints/post_processing/index.md
Cleaned and saved: checkpoints/post_processing/CapabilityStatement_plan_net.md

Processing complete: 7 files successfully cleaned, 0 failed


## Requirements Extraction

### Prompt-based Requirement Extraction

In [ ]:
#importlib.reload(reqs_extraction)

NameError: name 'reqs_extraction' is not defined

In [42]:
import reqs_extraction

In [43]:
reqs_extraction.run_requirements_extractor(
    '/Users/ceadams/Documents/onclaive/onclaive/full-ig/markdown_cleaned', 
    'checkpoints/requirements_extraction/markdown/plan-net', 
    'claude', 
    llm_clients)

INFO:root:Found markdown directory at /Users/ceadams/Documents/onclaive/onclaive/full-ig/markdown_cleaned
INFO:root:Found 242 markdown files
INFO:root:Processing with claude...
INFO:root:Starting processing with claude on directory: /Users/ceadams/Documents/onclaive/onclaive/full-ig/markdown_cleaned
INFO:root:Found 242 markdown files:
INFO:root:  - Location-HospLoc1.md
INFO:root:  - SearchParameter-location-endpoint.md
INFO:root:  - HealthcareService-HartfordOrthopedicServices.md
INFO:root:  - implementation.md
INFO:root:  - StructureDefinition-plannet-Organization-definitions.md
INFO:root:  - StructureDefinition-practitioner-qualification.profile.history.md
INFO:root:  - StructureDefinition-communication-proficiency-mappings.md
INFO:root:  - examples.md
INFO:root:  - StructureDefinition-plannet-InsurancePlan-examples.md
INFO:root:  - PractitionerRole-JoeSmithRole2.md
INFO:root:  - SearchParameter-healthcareservice-location.md
INFO:root:  - SearchParameter-practitionerrole-period.md
IN


Processing Implementation Guide with Claude...
This may take several minutes depending on the size of the Implementation Guide.


INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:root:Processing combined group of 15 files
INFO:root:Split combined content into 1 chunks
INFO:root:Processing chunk 1/1 of combined files
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:root:Processing combined group of 10 files
INFO:root:Split combined content into 1 chunks
INFO:root:Processing chunk 1/1 of combined files
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:root:Processing combined group of 8 files
INFO:root:Split combined content into 1 chunks
INFO:root:Processing chunk 1/1 of combined files
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:root:Processing combined group of 8 files
INFO:root:Split combined content into 1 chunks
INFO:root:Processing chunk 1/1 of combined files
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 


Processing complete!
Generated requirements document: checkpoints/requirements_extraction/markdown/plan-net/claude_reqs_list_v1_20250806_150005.md
Processed 242 files


## Requirement Refinement with LLM

In [24]:
import reqs_reviewer
importlib.reload(reqs_reviewer)

<module 'reqs_reviewer' from '/Users/ceadams/Documents/onclaive/onclaive/pipeline/reqs_reviewer.py'>

In [31]:
# Manual refinement with specific file
refinement_result = reqs_reviewer.refine_requirements(
    input_file="checkpoints/requirements_extraction/markdown/plan-net/claude_reqs_list_v1_20250805_133030.md",
    api_type="gpt",
    output_dir="checkpoints/revised_reqs_extraction",
    llm_client_instance=llm_clients
)

print(f"✅ Refined requirements saved to: {refinement_result['output_file']}")
print(f"📊 {refinement_result['original_requirements_count']} → {refinement_result['requirements_count']} requirements")

if refinement_result.get('warnings'):
    print("⚠️  Warnings:")
    for warning in refinement_result['warnings']:
        print(f"  - {warning}")

INFO:root:Prompt environment set up at: /Users/ceadams/Documents/onclaive/onclaive/prompts
INFO:root:Starting requirements refinement with gpt
INFO:root:Original requirements count: 399
INFO:root:Input size: 189679 characters, ~41931 tokens
INFO:root:Sending requirements to gpt for refinement...
INFO:root:Estimated input tokens: 42399
INFO:root:API input limit: 120000
INFO:root:API output limit: 8192
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Requirements refinement complete. Output saved to: checkpoints/revised_reqs_extraction/gpt_reqs_list_v2_20250805_150029.md
INFO:root:Refined 399 -> 52 requirements
INFO:root:Output size: 18828 characters, ~4449 tokens


✅ Refined requirements saved to: checkpoints/revised_reqs_extraction/gpt_reqs_list_v2_20250805_150029.md
📊 399 → 52 requirements
⚠️  Warnings:
  - Large reduction in requirements count (from 399 to 52). Verify this is expected.


In [ ]:
# Interactive refinement
refinement_result = reqs_reviewer.run_interactive_refinement()

## Requirement Downselection

In [41]:
import requirement_downselect
importlib.reload(requirement_downselect)
requirement_downselect.full_pass(
    md_files=["/Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250805_125844.md", "/Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250805_125935.md", "/Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250805_130022.md", "/Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250805_130106.md", "/Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250805_130905.md", "/Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250805_133736.md", "/Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250805_135555.md" ,"/Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250805_144850.md"]
    #rag_files=["checkpoints/requirements_extraction/RAG/raw_output.json"]
    )

/Users/ceadams/Documents/onclaive/onclaive/pipeline/requirement_downselect.py:23: SyntaxWarning: invalid escape sequence '\*'
  headings = list(re.finditer("\*\*\w+\*\*\:", split))
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-mpnet-base-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Final number of requirements: 30
Output saved in Markdown format in directory: checkpoints/requirements_downselect


## Test Plan Generation

In [8]:
import logging
llm_clients.logger.setLevel(logging.INFO)

In [9]:
import req_to_testplan
importlib.reload(req_to_testplan)

result = req_to_testplan.generate_consolidated_test_plan(
    llm_clients,
    'claude',  # 'claude' or 'gemini' or 'gpt'
    llm_clients.logger,
    "/Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250730_122703.md",
    "/Users/ceadams/Documents/onclaive/onclaive/full-ig/markdown7_cleaned/CapabilityStatement_plan_net.md",
    "Plan-Net IG",
    output_dir='checkpoints/testplan_generation'
)

print(f"Generated test plan with improved capability parsing: {result['test_plan_path']}")

INFO:llm_utils:Starting test plan generation with claude for Plan-Net IG
INFO:llm_utils:Parsed 19 requirements from /Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250730_122703.md
INFO:llm_utils:Parsed capability statement from /Users/ceadams/Documents/onclaive/onclaive/full-ig/markdown7_cleaned/CapabilityStatement_plan_net.md
INFO:llm_utils:Identifying group for requirement REQ-01 using claude...
INFO:llm_utils:Identifying group for requirement REQ-02 using claude...
INFO:llm_utils:Identifying group for requirement REQ-03 using claude...
INFO:llm_utils:Identifying group for requirement REQ-04 using claude...
INFO:llm_utils:Identifying group for requirement REQ-05 using claude...
INFO:llm_utils:Identifying group for requirement REQ-06 using claude...
INFO:llm_utils:Identifying group for requirement REQ-07 using claude...
INFO:llm_utils:Identifying group for requirement REQ-08 using claude...
INFO:llm_utils:Identifying group

Generated test plan with improved capability parsing: checkpoints/testplan_generation/claude_test_plan_20250730_123104.md


In [ ]:
capability_statement = req_to_testplan.parse_capability_statement_by_chunks(
    "/Users/ceadams/Documents/onclaive/onclaive/full-ig/markdown7_cleaned/CapabilityStatement_plan_net.md"
)

print(f"\nCapability Statement Analysis:")
print(f"- Found {len(capability_statement['resource_list'])} resources: {capability_statement['resource_list']}")
print(f"- General capabilities text length: {len(capability_statement['general_capabilities'])} characters")



Capability Statement Analysis:
- Found 8 resources: ['Endpoint', 'HealthcareService', 'InsurancePlan', 'Location', 'Organization', 'OrganizationAffiliation', 'Practitioner', 'PractitionerRole']
- General capabilities text length: 1113 characters


In [ ]:
sample_requirement = {
    'summary': 'Organization search by name',
    'description': 'The Plan-Net design is based around the following types of searches: Organization by Name - Example: Montgomery Cardiology or CVS - Focal Resource and Field: Organization.name - Qualifications of Search: Location, network, specialty.',
    'verification': 'Automatically testable',
    'actor': 'Health Plan API'
}

relevant_info = req_to_testplan.extract_relevant_capability_chunks(sample_requirement, capability_statement)
print(f"\nSample requirement matching:")
print(f"- Relevant capability info length: {len(relevant_info)} characters")
print(f"- Preview: {relevant_info}...")


Sample requirement matching:
- Relevant capability info length: 6082 characters
- Preview: ### Applicable Capability Statement Information

#### Relevant Resource Capabilities

#### Location

Conformance Expectation: **SHALL**

Supported Profiles:

 [Plan\-Net Location](StructureDefinition-plannet-Location.html)

Reference Policy: `resolves`

Profile Interaction Summary:

* **SHALL** support 
 `search-type`, 
 `read`.
* **SHOULD** support 
 `vread`.

Fetch and Search Criteria:

* A Server **SHALL** be capable of returning a Location resource using:

`GET [base]/Location/[id]`

* A Server **SHOULD** be capable of returning a Location resource using:

`GET [base]/Location/[id]/_history/vid`

 * A Server **SHALL** be capable of supporting the following \_includes:

 Location:endpoint \- `GET [base]/Location?[parameter=value]&_include=Location:endpoint`  

 Location:organization \- `GET [base]/Location?[parameter=value]&_include=Location:organization`  

 Location:partof \- `GET [base]/Lo

### RAG Approach

In [17]:
import warnings
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Set logging level to reduce noise
import logging
logging.getLogger("urllib3.connectionpool").setLevel(logging.ERROR)
logging.getLogger("backoff").setLevel(logging.ERROR)

import req_to_testplan_rag
importlib.reload(req_to_testplan_rag)

req_to_testplan_rag.clear_capability_collection("capability_statements")


Deleted existing collection: capability_statements


ERROR:backoff:Giving up send_request(...) after 4 tries (requests.exceptions.SSLError: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)'))))


In [18]:

req_to_testplan_rag.generate_consolidated_test_plan(
    llm_clients, 
    'claude', 
    llm_clients.logger, 
    "/Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250730_122703.md", 
    "../full-ig/markdown7_cleaned/CapabilityStatement_plan_net.md", 
    "Plan-Net IG",
    output_dir='checkpoints/testplan_generation',
    verbose=True)

INFO:llm_utils:Starting test plan generation with claude for Plan-Net IG
INFO:llm_utils:Parsed 19 requirements from /Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250730_122703.md
INFO:llm_utils:Initialized capability collection from ../full-ig/markdown7_cleaned/CapabilityStatement_plan_net.md
INFO:llm_utils:Identifying group for requirement REQ-01 using claude...
INFO:llm_utils:Identifying group for requirement REQ-02 using claude...
ERROR:backoff:Giving up send_request(...) after 4 tries (requests.exceptions.SSLError: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)'))))
INFO:llm_utils:Identifying group for requirement REQ-03 using claude...
INFO:llm_utils:Identifying group for requirement REQ-04 using claude...
INFO:


Processing Group: General/Cross-Resource (15 requirements)

Processing REQ-03: Health Plan API Must Support data population capability

RAG RETRIEVAL FOR REQ-03
Query: Health Plan API Must Support data population capability "Health Plan API actors SHALL be capable of populating all Must Support data elements as part of the query results." Health Plan API FHIR SHALL
Searching for 5 most relevant capability chunks...

Found 5 matching chunks:

  Match 1 (distance: 0.7221184968948364):
  Length: 3036 chars
  Preview: FHIR Resource Detail: #### HealthcareService

Conformance Expectation: **SHALL**

Supported Profiles...
  ...lthcareService?_lastUpdated=[_lastUpdated]` |

---

  Match 2 (distance: 0.7407464981079102):
  Length: 3288 chars
  Preview: FHIR Resource/Component: ### RESTful Capabilities by Resource/Profile:

**Summary of Search Criteria...
  ...nerRole:network, PractitionerRole:endpoint |  |  |

  Match 3 (distance: 0.8931890726089478):
  Length: 3225 chars
  Preview: FHIR Reso

ERROR:backoff:Giving up send_request(...) after 4 tries (requests.exceptions.SSLError: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)'))))


Completed test specification for REQ-03


INFO:llm_utils:Processing requirement for group 'General/Cross-Resource': REQ-04
INFO:llm_utils:Generating test specification for REQ-04 using claude...


...

INFO:llm_utils:Processing requirement for group 'Security': REQ-18
INFO:llm_utils:Generating test specification for REQ-18 using claude...


Completed test specification for REQ-02

Processing Group: Security (1 requirements)

Processing REQ-18: Plan-Net Server unauthorized request rejection

RAG RETRIEVAL FOR REQ-18
Query: Plan-Net Server unauthorized request rejection "A server SHALL reject any unauthorized requests by returning an `HTTP 401` unauthorized response code." Plan-Net Server FHIR SHALL
Searching for 5 most relevant capability chunks...

Found 5 matching chunks:

  Match 1 (distance: 0.5878045558929443):
  Length: 1169 chars
  Preview: FHIR Resource/Component: ### FHIR RESTful Capabilities

The Plan\-Net Server **SHALL**:

1. Support ...
  ...eturning an `HTTP 401` unauthorized response code.

  Match 2 (distance: 0.7881736159324646):
  Length: 996 chars
  Preview: FHIR Major Section: ## CapabilityStatement: Plan\-Net CapabilityStatement

| *Official URL*: http://...
  ...openapi.json) \| [Download](plan-net.openapi.json)

  Match 3 (distance: 0.9208030700683594):
  Length: 802 chars
  Preview: FHIR Major Secti

INFO:llm_utils:Consolidated test plan saved to checkpoints/testplan_generation/claude_test_plan_20250730_140520.md


Completed test specification for REQ-18


{'requirements_count': 19,
 'group_count': 4,
 'test_plan_path': 'checkpoints/testplan_generation/claude_test_plan_20250730_140520.md'}

In [24]:
import chromadb
from chromadb.utils import embedding_functions
from datetime import datetime
from pathlib import Path


def inspect_capability_chunks(output_file=None):
    """
    Inspect all chunks stored in the ChromaDB collection and save to markdown file
    """
    # Connect to ChromaDB
    chroma_client = chromadb.Client()
    sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
        model_name="all-mpnet-base-v2"
    )
    
    try:
        # Get the existing collection
        collection = chroma_client.get_collection('capability_statements', embedding_function=sentence_transformer_ef)
        
        # Get all documents
        all_docs = collection.get()
        
        # Create output file name if not provided
        if output_file is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_file = f"capability_chunks_analysis_{timestamp}.md"
        
        # Prepare markdown content
        markdown_content = []
        
        # Header
        markdown_content.append("# Capability Statement Chunking Analysis")
        markdown_content.append(f"*Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*\n")
        
        # Summary statistics
        chunk_lengths = [len(doc) for doc in all_docs['documents']]
        markdown_content.append("## 📊 Summary Statistics")
        markdown_content.append(f"- **Total chunks**: {len(all_docs['documents'])}")
        markdown_content.append(f"- **Average chunk length**: {sum(chunk_lengths) / len(chunk_lengths):.1f} characters")
        markdown_content.append(f"- **Shortest chunk**: {min(chunk_lengths)} characters")
        markdown_content.append(f"- **Longest chunk**: {max(chunk_lengths)} characters\n")
        
        # Chunk type analysis
        chunk_types = {
            'Resource sections': 0,
            'Table sections': 0,
            'General sections': 0,
            'Other': 0
        }
        
        for doc in all_docs['documents']:
            if 'FHIR' in doc and 'Resource:' in doc:
                chunk_types['Resource sections'] += 1
            elif 'FHIR Capability Table:' in doc:
                chunk_types['Table sections'] += 1
            elif 'FHIR Capability Statement:' in doc:
                chunk_types['General sections'] += 1
            else:
                chunk_types['Other'] += 1
        
        markdown_content.append("## 🔍 Chunk Type Analysis")
        for chunk_type, count in chunk_types.items():
            markdown_content.append(f"- **{chunk_type}**: {count} chunks")
        markdown_content.append("")
        
        # Table of contents
        markdown_content.append("## 📋 Table of Contents")
        for i in range(len(all_docs['documents'])):
            doc = all_docs['documents'][i]
            # Create a short preview for TOC
            preview = doc[:50].replace('\n', ' ').strip()
            if len(doc) > 50:
                preview += "..."
            markdown_content.append(f"- [Chunk {i+1}](#chunk-{i+1}) ({len(doc)} chars): {preview}")
        markdown_content.append("")
        
        # Detailed chunks
        markdown_content.append("## 📄 Detailed Chunk Analysis")
        
        for i, doc in enumerate(all_docs['documents']):
            # Chunk header
            markdown_content.append(f"### Chunk {i+1}")
            markdown_content.append(f"**Length**: {len(doc)} characters  ")
            markdown_content.append(f"**ID**: {all_docs['ids'][i]}  ")
            
            # Determine chunk type
            chunk_type = "Other"
            if 'FHIR' in doc and 'Resource:' in doc:
                chunk_type = "Resource Section"
            elif 'FHIR Capability Table:' in doc:
                chunk_type = "Table Section"
            elif 'FHIR Capability Statement:' in doc:
                chunk_type = "General Section"
            
            markdown_content.append(f"**Type**: {chunk_type}\n")
            
            # Content in code block for better formatting
            markdown_content.append("**Content:**")
            markdown_content.append("```")
            markdown_content.append(doc)
            markdown_content.append("```\n")
            
            markdown_content.append("---\n")
        
        # Write to file
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write('\n'.join(markdown_content))
        
        print(f"✅ Chunk analysis saved to: {output_file}")
        print(f"📊 Total chunks analyzed: {len(all_docs['documents'])}")
        
        # Also print summary to console
        print(f"\n📋 QUICK SUMMARY:")
        print(f"   Total chunks: {len(all_docs['documents'])}")
        for chunk_type, count in chunk_types.items():
            print(f"   {chunk_type}: {count}")
            
        return all_docs, output_file
        
    except Exception as e:
        print(f"❌ Error inspecting collection: {e}")
        print("Make sure you've run the script first to create the collection!")
        return None, None

def quick_chunk_preview(output_file=None, num_chunks=5):
    """Create a shorter markdown file with just preview of first few chunks"""
    chroma_client = chromadb.Client()
    sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
        model_name="all-mpnet-base-v2"
    )
    
    try:
        collection = chroma_client.get_collection('capability_statements', embedding_function=sentence_transformer_ef)
        all_docs = collection.get()
        
        if output_file is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_file = f"capability_chunks_preview_{timestamp}.md"
        
        markdown_content = []
        markdown_content.append("# Capability Statement Chunks Preview")
        markdown_content.append(f"*Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*\n")
        markdown_content.append(f"Showing first {num_chunks} chunks out of {len(all_docs['documents'])} total\n")
        
        for i in range(min(num_chunks, len(all_docs['documents']))):
            doc = all_docs['documents'][i]
            markdown_content.append(f"## Chunk {i+1}")
            markdown_content.append(f"**Length**: {len(doc)} characters\n")
            markdown_content.append("```")
            markdown_content.append(doc)
            markdown_content.append("```\n")
        
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write('\n'.join(markdown_content))
        
        print(f"✅ Chunk preview saved to: {output_file}")
        return output_file
        
    except Exception as e:
        print(f"Error: {e}")
        return None

# Run the full analysis and save to markdown
chunks_data, analysis_file = inspect_capability_chunks()

✅ Chunk analysis saved to: capability_chunks_analysis_20250702_135655.md
📊 Total chunks analyzed: 13

📋 QUICK SUMMARY:
   Total chunks: 13
   Resource sections: 0
   Table sections: 0
   General sections: 0
   Other: 13


2025-07-02 13:57:00,559 - backoff - ERROR - Giving up send_request(...) after 4 tries (requests.exceptions.SSLError: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)'))))


## Test Kit Generation

In [49]:
import plan_to_tests
import importlib
importlib.reload(plan_to_tests)

plan_to_tests.generate_inferno_test_kit(
    llm_clients,
    'claude',
    'checkpoints/testplan_generation/claude_test_plan_20250730_140520.md',
    #'../test_kit_dev/inferno-guidance.md',
    output_dir='checkpoints/testkit_generation',
    expected_actors=["Health Plan API Actor", "Application Actor"]
)

/Users/ceadams/Documents/onclaive/onclaive/pipeline/plan_to_tests.py:775: SyntaxWarning: invalid escape sequence '\A'
  api_type,
INFO:plan_to_tests:Starting Inferno test generation with claude for PlanNet
INFO:plan_to_tests:Parsed test plan into 4 sections
INFO:plan_to_tests:Found 19 total requirements
INFO:plan_to_tests:Loaded Inferno DSL guidance
INFO:plan_to_tests:Processing section: General/Cross-Resource with 15 requirements
INFO:plan_to_tests:Generating tests for section: General/Cross-Resource
INFO:plan_to_tests:Sending 161 tokens to claude API (limit: 16000)
INFO:plan_to_tests:Attempting to generate tests for entire section: General/Cross-Resource


Found 19 potential requirements
Processing requirement: REQ-03
Added requirement REQ-03 to section General/Cross-Resource
Processing requirement: REQ-04
Added requirement REQ-04 to section General/Cross-Resource
Processing requirement: REQ-05
Added requirement REQ-05 to section General/Cross-Resource
Processing requirement: REQ-06
Added requirement REQ-06 to section General/Cross-Resource
Processing requirement: REQ-07
Added requirement REQ-07 to section General/Cross-Resource
Processing requirement: REQ-08
Added requirement REQ-08 to section General/Cross-Resource
Processing requirement: REQ-09
Added requirement REQ-09 to section General/Cross-Resource
Processing requirement: REQ-10
Added requirement REQ-10 to section General/Cross-Resource
Processing requirement: REQ-11
Added requirement REQ-11 to section General/Cross-Resource
Processing requirement: REQ-12
Added requirement REQ-12 to section General/Cross-Resource
Processing requirement: REQ-13
Added requirement REQ-13 to section G

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:plan_to_tests:Generating test for requirement: REQ-03
INFO:plan_to_tests:Requirement REQ-03: Sending 1275 tokens to claude API (limit: 16000)
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:plan_to_tests:Successfully generated test for requirement: REQ-03
INFO:plan_to_tests:Generating test for requirement: REQ-04
INFO:plan_to_tests:Requirement REQ-04: Sending 1307 tokens to claude API (limit: 16000)
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:plan_to_tests:Successfully generated test for requirement: REQ-04
INFO:plan_to_tests:Generating test for requirement: REQ-05
INFO:plan_to_tests:Requirement REQ-05: Sending 1313 tokens to claude API (limit: 16000)
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:plan_to_tests:Successfully generated test for requirement: REQ-05
IN

KeyboardInterrupt: 